In [15]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import pickle

In [16]:
data=pd.read_csv('data.csv')
data.head() 

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [17]:
data.drop(['Surname','RowNumber','CustomerId'],axis=1,inplace=True)

In [18]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
data['Gender']=le.fit_transform(data['Gender'])
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [19]:
## Onehot encode 'Geography
from sklearn.preprocessing import OneHotEncoder
onehot_encoder_geo=OneHotEncoder()
geo_encoder=onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoder


onehot_encoder_geo.get_feature_names_out(['Geography'])

geo_encoded_df=pd.DataFrame(geo_encoder,columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

data=pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [20]:
## Save the encoders and sscaler
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(le,file)

with open('onehot_encoder_geo.pkl','wb') as file:
    pickle.dump(onehot_encoder_geo,file)


In [21]:
X=data.drop('Exited',axis=1)
y=data['Exited']

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
X_train_scaled=scaler.fit_transform(X_train)
X_test_scaled=scaler.transform(X_test)

In [22]:
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

# ANN

In [23]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

### Build ANN Model

In [24]:
model=Sequential([
    Dense(64,activation='relu',input_shape=(X_train_scaled.shape[1],)), ##HL1 connected with input layer
    Dense(32,activation='relu'),  ##HL2
    Dense(1,activation='sigmoid')  ##Output Layer
])

### Compile the model

In [ ]:
opt=tf.keras.optimizers.Adam(learning_rate=0.001)  
#when using Adam optimizer, it is better to set learning rate explicitly

In [ ]:
model.compile(optimizer=opt,loss='binary_crossentropy',metrics=['accuracy'])

In [30]:
logdir="logs/fit" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_calllback=TensorBoard(log_dir=logdir, histogram_freq=1) 

In [33]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(
    monitor='val_loss',    # what metric to monitor
    patience=10,            # how many epochs to wait before stopping
    restore_best_weights=True  # revert to best model before overfitting
)


### Train the model


In [34]:
history=model.fit(X_train_scaled,y_train,validation_data=(X_test_scaled,y_test),epochs=100,
                  callbacks=[early_stop,tensorflow_calllback]
                )

Epoch 1/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3252 - accuracy: 0.8659 - val_loss: 0.3401 - val_accuracy: 0.8635
Epoch 2/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3232 - accuracy: 0.8670 - val_loss: 0.3425 - val_accuracy: 0.8575
Epoch 3/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3219 - accuracy: 0.8680 - val_loss: 0.3421 - val_accuracy: 0.8590
Epoch 4/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3201 - accuracy: 0.8665 - val_loss: 0.3368 - val_accuracy: 0.8660
Epoch 5/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3182 - accuracy: 0.8666 - val_loss: 0.3400 - val_accuracy: 0.8615
Epoch 6/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3175 - accuracy: 0.8669 - val_loss: 0.3405 - val_accuracy: 0.8620
Epoch 7/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3156 - accuracy: 0.8691 - val_loss: 0.3448 - val_accuracy: 0.8565

In [35]:
model.save('model.h5')

c:\Users\Admin\Desktop\ANN\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
